# Library importation

In [1]:
import numpy as np
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import re
import ast 
import json

# Recupération de la liste des brevets marqués

In [2]:
list_patents = pd.read_csv('./Data/patents_number.csv',index_col=None)

In [3]:
L = list(list_patents.patent_number.values)

In [4]:
df = pd.DataFrame(columns=['patent_number', 'patent_date',
                           'patent_processing_time', 'ipc_section', 'ipc_class', 'assignee_id', ])

# Récuperation des informations des brevets marqués

In [5]:
i=0
while i < (len(L)-700):    
    token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
    token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + str(L[i:i+700]) + token2
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["patents"]
    df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
    i+=700
    print("{}% done".format(i/len(L)*100))
token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
url = token1 + str(L[i:]) + token2
r = requests.get(url)
html=r.text
html = html.replace("null","None")
Dict = ast.literal_eval(html)["patents"]
df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
df = df.reset_index(drop=True)

2.8957928267074835% done
5.791585653414967% done
8.687378480122451% done
11.583171306829934% done
14.478964133537417% done
17.374756960244902% done
20.270549786952387% done
23.166342613659868% done
26.06213544036735% done
28.957928267074834% done
31.853721093782315% done
34.749513920489804% done
37.64530674719729% done
40.54109957390477% done
43.43689240061225% done
46.332685227319736% done
49.22847805402722% done
52.1242708807347% done
55.02006370744219% done
57.91585653414967% done
60.81164936085715% done
63.70744218756463% done
66.60323501427212% done
69.49902784097961% done
72.39482066768709% done
75.29061349439458% done
78.18640632110206% done
81.08219914780955% done
83.97799197451702% done
86.8737848012245% done
89.769577627932% done
92.66537045463947% done
95.56116328134696% done
98.45695610805444% done


In [6]:
df["marked"]=[1]*len(df)

In [7]:
assignee_id=[]

for index,j in enumerate(df.assignees.values):
    assignees=[]
    for element in j:
        assignees.append(element["assignee_id"])
    if len(assignees)>1:
        assignee_id.append(assignees)
    else:
        assignee_id.append(assignees[0])
df["assignee_id"] = assignee_id
df = df.drop(['assignees'],axis=1)

In [8]:
df.patent_number=df.patent_number.astype(str)

In [9]:
df

,patent_number,patent_date,patent_processing_time,ipc_section,ipc_class,assignee_id,IPCs,wipos,marked
0,5387603,1995-02-07,433,NaN,NaN,5330279e-a21e-438c-80d1-7e6ae793c092,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
1,5771295,1998-06-23,552,NaN,NaN,2986e46e-7937-477f-9bc5-1f5eec05183f,"[{'ipc_section': 'H', 'ipc_class': '04'}]",[{'wipo_field_id': '2'}],1
2,5585587,1996-12-17,832,NaN,NaN,f7152515-d420-42d9-a7b6-3a2796bc5d10,"[{'ipc_section': 'G', 'ipc_class': '10'}]","[{'wipo_field_id': '34'}, {'wipo_field_id': '2...",1
3,5321490,1994-06-14,568,NaN,NaN,ec72750e-f789-4749-91f7-6292f7c6fda7,"[{'ipc_section': 'G', 'ipc_class': '01'}]","[{'wipo_field_id': '10'}, {'wipo_field_id': '1...",1
4,5750497,1998-05-12,1161,NaN,NaN,9a3afa40-66e5-4921-b831-64287f2052e0,"[{'ipc_section': 'C', 'ipc_class': '07'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
...,...,...,...,...,...,...,...,...,...
23905,10299691,2019-05-28,225,NaN,NaN,1b2d67d2-f1f9-4c29-8cd1-a5e75ba27be7,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...",[{'wipo_field_id': '13'}],1
23906,10292890,2019-05-21,966,NaN,NaN,f0997ba8-1ec2-4c0c-aaea-d80066a49317,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1
23907,10299968,2019-05-28,343,NaN,NaN,e5fc32c3-a913-455a-94ce-9932d30eefff,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '13'}, {'wipo_field_id': '6'}]",1
23908,10149616,2018-12-11,2133,NaN,NaN,4eb39673-7f59-4c45-bf6c-2275dccf9048,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1


# Constructing the list of assignees

In [10]:
assignees = []
for el in df.assignee_id:
    if type(el)==list:
        for i in el:
            if i not in assignees:
                assignees.append(i)
    elif el!=None and el not in assignees:
        assignees.append(el)

In [11]:
L_assignees = []
k=0
while k < len(assignees)-100:
    L_assignees.append(json.dumps(assignees[k:k+100]))
    k+=100
L_assignees.append(json.dumps(assignees[k:]))

# Récuperation des brevets non marqués de ces entreprises 

In [12]:
L_unmarked=[]
#df_list_assignee = pd.DataFrame()
for i, el in enumerate(L_assignees):
    token1 = 'https://api.patentsview.org/assignees/query?q={"assignee_id":'
    token2 = '}&f=["patent_number","assignee_id"]&o={"page":'
    token3 = ',"per_page":10000}'
    url = token1 + el + token2 + str(1) + token3
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["assignees"]
    for d in Dict:
        patents = d["patents"]
        for patent in patents:
            if patent["patent_number"].isnumeric():
                L_unmarked.append(int(patent["patent_number"]))
            else:
                L_unmarked.append("{}".format(patent["patent_number"]))
    #     df_list_assignee = pd.concat([df_list_assignee,pd.read_json(json.dumps(Dict),orient='records')])
    print(i*100)
print(L_unmarked)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
L_unmarked = list(set(L_unmarked)-set(df.patent_number.values))

In [14]:
i=0
while i < (len(L_unmarked)-600):    
    token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
    token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + str(L_unmarked[i:i+600]) + token2
    url = url.replace("\'",'"')
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["patents"]
    df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
    i+=600
    print("{}% done".format(i/len(L_unmarked)*100))
token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
token2 = '}&f=["patent_number","patent_date","patent_processing_time","assignee_id","ipc_section","ipc_class","wipo_field_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
url = token1 + str(L_unmarked[i:]) + token2
r = requests.get(url)
html=r.text
html = html.replace("null","None")
Dict = ast.literal_eval(html)["patents"]
df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
df = df.reset_index(drop=True)

0.03131002719275862% done
0.06262005438551724% done
0.09393008157827586% done
0.12524010877103448% done
0.15655013596379308% done
0.1878601631565517% done
0.21917019034931032% done
0.25048021754206895% done
0.28179024473482756% done
0.31310027192758616% done
0.3444102991203448% done
0.3757203263131034% done
0.4070303535058621% done
0.43834038069862064% done
0.4696504078913793% done
0.5009604350841379% done
0.5322704622768964% done
0.5635804894696551% done
0.5948905166624138% done
0.6262005438551723% done
0.657510571047931% done
0.6888205982406896% done
0.7201306254334482% done
0.7514406526262069% done
0.7827506798189655% done
0.8140607070117242% done
0.8453707342044826% done
0.8766807613972413% done
0.9079907885899999% done
0.9393008157827586% done
0.970610842975517% done
1.0019208701682758% done
1.0332308973610345% done
1.064540924553793% done
1.0958509517465516% done
1.1271609789393102% done
1.1584710061320689% done
1.1897810333248275% done
1.2210910605175862% done
1.2524010877103446

10.582789191152413% done
10.61409921834517% done
10.64540924553793% done
10.676719272730688% done
10.708029299923446% done
10.739339327116205% done
10.770649354308965% done
10.801959381501723% done
10.833269408694482% done
10.86457943588724% done
10.895889463079998% done
10.927199490272757% done
10.958509517465515% done
10.989819544658275% done
11.021129571851034% done
11.052439599043792% done
11.08374962623655% done
11.11505965342931% done
11.146369680622067% done
11.177679707814827% done
11.208989735007584% done
11.240299762200344% done
11.271609789393104% done
11.30291981658586% done
11.334229843778619% done
11.365539870971379% done
11.396849898164136% done
11.428159925356896% done
11.459469952549654% done
11.490779979742411% done
11.522090006935171% done
11.553400034127929% done
11.584710061320688% done
11.616020088513448% done
11.647330115706206% done
11.678640142898965% done
11.709950170091723% done
11.74126019728448% done
11.77257022447724% done
11.803880251669998% done
11.83519

20.977718219148272% done
21.009028246341032% done
21.04033827353379% done
21.07164830072655% done
21.102958327919307% done
21.134268355112066% done
21.165578382304826% done
21.196888409497582% done
21.22819843669034% done
21.2595084638831% done
21.29081849107586% done
21.322128518268617% done
21.353438545461376% done
21.384748572654136% done
21.41605859984689% done
21.447368627039655% done
21.47867865423241% done
21.50998868142517% done
21.54129870861793% done
21.572608735810686% done
21.603918763003445% done
21.635228790196205% done
21.666538817388965% done
21.69784884458172% done
21.72915887177448% done
21.76046889896724% done
21.791778926159996% done
21.823088953352755% done
21.854398980545515% done
21.885709007738274% done
21.91701903493103% done
21.94832906212379% done
21.97963908931655% done
22.010949116509305% done
22.04225914370207% done
22.073569170894825% done
22.104879198087584% done
22.136189225280344% done
22.1674992524731% done
22.19880927966586% done
22.23011930685862% d

31.435267301529652% done
31.466577328722412% done
31.497887355915168% done
31.529197383107928% done
31.560507410300687% done
31.591817437493447% done
31.623127464686203% done
31.654437491878962% done
31.68574751907172% done
31.717057546264478% done
31.748367573457237% done
31.779677600649997% done
31.810987627842756% done
31.842297655035512% done
31.873607682228272% done
31.90491770942103% done
31.936227736613787% done
31.967537763806547% done
31.998847790999307% done
32.030157818192066% done
32.061467845384826% done
32.092777872577585% done
32.124087899770345% done
32.1553979269631% done
32.18670795415586% done
32.218017981348616% done
32.249328008541376% done
32.280638035734135% done
32.311948062926895% done
32.343258090119654% done
32.37456811731241% done
32.40587814450517% done
32.437188171697926% done
32.468498198890686% done
32.499808226083445% done
32.531118253276205% done
32.562428280468964% done
32.59373830766172% done
32.625048334854476% done
32.656358362047236% done
32.68766

41.986746465489304% done
42.018056492682064% done
42.04936651987482% done
42.08067654706758% done
42.11198657426034% done
42.1432966014531% done
42.174606628645854% done
42.205916655838614% done
42.23722668303137% done
42.26853671022413% done
42.29984673741689% done
42.33115676460965% done
42.36246679180241% done
42.393776818995164% done
42.425086846187924% done
42.45639687338068% done
42.48770690057344% done
42.5190169277662% done
42.55032695495896% done
42.58163698215172% done
42.612947009344474% done
42.64425703653723% done
42.67556706373% done
42.70687709092275% done
42.73818711811551% done
42.76949714530827% done
42.80080717250103% done
42.83211719969378% done
42.86342722688655% done
42.89473725407931% done
42.92604728127206% done
42.95735730846482% done
42.98866733565758% done
43.01997736285034% done
43.0512873900431% done
43.08259741723586% done
43.11390744442862% done
43.14521747162137% done
43.17652749881413% done
43.20783752600689% done
43.23914755319965% done
43.270457580392

52.63215571102724% done
52.66346573822% done
52.69477576541275% done
52.72608579260552% done
52.75739581979827% done
52.78870584699102% done
52.82001587418379% done
52.85132590137655% done
52.8826359285693% done
52.91394595576207% done
52.94525598295482% done
52.97656601014757% done
53.00787603734034% done
53.0391860645331% done
53.07049609172586% done
53.10180611891862% done
53.13311614611137% done
53.164426173304136% done
53.19573620049689% done
53.22704622768965% done
53.25835625488241% done
53.28966628207517% done
53.32097630926792% done
53.352286336460686% done
53.38359636365344% done
53.4149063908462% done
53.44621641803896% done
53.47752644523172% done
53.508836472424484% done
53.54014649961724% done
53.57145652680999% done
53.602766554002756% done
53.63407658119551% done
53.66538660838827% done
53.696696635581034% done
53.72800666277379% done
53.75931668996654% done
53.790626717159306% done
53.82193674435206% done
53.85324677154482% done
53.884556798737584% done
53.915866825930

63.246254929372405% done
63.27756495656517% done
63.308874983757924% done
63.34018501095068% done
63.37149503814344% done
63.402805065336196% done
63.434115092528955% done
63.46542511972172% done
63.496735146914475% done
63.52804517410723% done
63.559355201299994% done
63.590665228492746% done
63.62197525568551% done
63.65328528287827% done
63.684595310071025% done
63.71590533726379% done
63.747215364456544% done
63.778525391649296% done
63.80983541884206% done
63.84114544603482% done
63.872455473227575% done
63.90376550042034% done
63.935075527613094% done
63.966385554805846% done
63.99769558199861% done
64.02900560919137% done
64.06031563638413% done
64.09162566357689% done
64.12293569076965% done
64.15424571796241% done
64.18555574515517% done
64.21686577234792% done
64.24817579954069% done
64.27948582673343% done
64.3107958539262% done
64.34210588111895% done
64.37341590831171% done
64.40472593550447% done
64.43603596269723% done
64.46734598988999% done
64.49865601708275% done
64.5

KeyboardInterrupt: 

In [16]:
df = df.reset_index(drop=True)

In [17]:
df

,patent_number,patent_date,patent_processing_time,ipc_section,ipc_class,assignee_id,IPCs,wipos,marked,assignees
0,5387603,1995-02-07,433,NaN,NaN,5330279e-a21e-438c-80d1-7e6ae793c092,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1.0,NaN
1,5771295,1998-06-23,552,NaN,NaN,2986e46e-7937-477f-9bc5-1f5eec05183f,"[{'ipc_section': 'H', 'ipc_class': '04'}]",[{'wipo_field_id': '2'}],1.0,NaN
2,5585587,1996-12-17,832,NaN,NaN,f7152515-d420-42d9-a7b6-3a2796bc5d10,"[{'ipc_section': 'G', 'ipc_class': '10'}]","[{'wipo_field_id': '34'}, {'wipo_field_id': '2...",1.0,NaN
3,5321490,1994-06-14,568,NaN,NaN,ec72750e-f789-4749-91f7-6292f7c6fda7,"[{'ipc_section': 'G', 'ipc_class': '01'}]","[{'wipo_field_id': '10'}, {'wipo_field_id': '1...",1.0,NaN
4,5750497,1998-05-12,1161,NaN,NaN,9a3afa40-66e5-4921-b831-64287f2052e0,"[{'ipc_section': 'C', 'ipc_class': '07'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1302505,6636893,2003-10-21,1118,NaN,NaN,NaN,"[{'ipc_section': 'G', 'ipc_class': '06'}]","[{'wipo_field_id': '1'}, {'wipo_field_id': '7'...",NaN,[{'assignee_id': 'f6b7446d-4090-4a4a-a7d2-3262...
1302506,10831331,2020-11-10,977,NaN,NaN,NaN,"[{'ipc_section': 'G', 'ipc_class': '6'}]","[{'wipo_field_id': '6'}, {'wipo_field_id': '7'}]",NaN,[{'assignee_id': 'ee0168d9-bf74-4513-bfbf-e73c...
1302507,10831407,2020-11-10,649,NaN,NaN,NaN,"[{'ipc_section': 'G', 'ipc_class': '6'}]",[{'wipo_field_id': '6'}],NaN,[{'assignee_id': '36bbeb70-9f21-4de3-8f53-01b9...
1302508,6636928,2003-10-21,1341,NaN,NaN,NaN,"[{'ipc_section': 'G', 'ipc_class': '06'}]",[{'wipo_field_id': '6'}],NaN,[{'assignee_id': '4acf646c-71dd-45a9-8c15-7857...


In [18]:
df.to_csv('./to_clean.csv',index=False)

# Nettoyage du DataFrame

In [49]:
ipc_section=[]
ipc_class=[]
for index,j in enumerate(df.IPCs.values):
    ipc = j
    for ele in ipc:
    print(ipc)

[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'C', 'ipc_class': '07'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '10'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'C', 'ipc_class': '07'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'B', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'B', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'C', 'ipc_class': '12'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '09'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '0

[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '07'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'B', 'ipc_class': '67'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'C', 'ipc_class': '12'}]
[{'ipc_section': 'A', 'ipc_class': '23'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'B', 'ipc_class': '65'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '02'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'B', 'ipc_class':

[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '23'}]
[{'ipc_section': 'H', 'ipc_class': '02'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'D', 'ipc_class': '06'}]
[{'ipc_section': 'B', 'ipc_class': '21'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '05'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '23'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'B', 'ipc_class': '23'}]
[{'ipc_section': 'G', 'ipc_class': '02'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'B', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '23'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'C', 'ipc_class':

[{'ipc_section': 'A', 'ipc_class': '63'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '02'}]
[{'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'G', 'ipc_class': '02'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'F', 'ipc_class': '16'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '01'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': 

[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'B', 'ipc_class': '29'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '10'}]
[{'ipc_section': 'F', 'ipc_class': '16'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'F', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'C', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '08'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '07'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '01'}, {'ipc_section': 'G', 'ipc_class': '02'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'H', 'ipc_class': '05'}]
[{'ipc_section': 'B', 'ipc_class': '05'}, {'ipc_section': 'A', 'ipc_class': '62'}]
[{'ipc_section': 'F', 'ipc_class': '16'

...

[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'B', 'ipc_class': '27'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'B', 'ipc_class': '60'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '08'}, {'ipc_section': 'G', 'ipc_class': '01'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'A', 'ipc_class': '63'}, {'ipc_section': 'G', 'ipc_class': '07'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '06'}

[{'ipc_section': 'H', 'ipc_class': '04'}, {'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'B', 'ipc_class': '65'}]
[{'ipc_section': 'B', 'ipc_class': '23'}, {'ipc_section': 'F', 'ipc_class': '16'}]
[{'ipc_section': 'G', 'ipc_class': '09'}, {'ipc_section': 'B', 'ipc_class': '23'}]
[{'ipc_section': 'G', 'ipc_class': '09'}, {'ipc_section': 'B', 'ipc_class': '23'}]
[{'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'B', 'ipc_class': '62'}, {'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'B', 'ipc_class': '66'}]
[{'ipc_section': 'F', 'ipc_class': '16'}, {'ipc_section': 'F', 'ipc_class': '28'}, {

[{'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'H', 'ipc_class': '04'}, {'ipc_section': 'G', 'ipc_class': '02'}, {'ipc_section': 'G', 'ipc_class': '03'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '10'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'H', 'ipc_class': '04'}, {'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'E', 'ipc_class': '04'}, {'ipc_section': 'B', 'ipc_class': '64'}, {'ipc_section': 'B', 'ipc_class': '32'}]
[{'ipc_section': 'G', 'ipc_class': '09'}, {'ipc_section': 'H', 'ipc_class': '04'}, {'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'H', 'ipc_class': '05'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'H', 'ipc_class': '01'}]
[{'ipc_section': 'C', 'ipc_class': '11'}]
[{'ipc_section': 'A', 'ipc_class': '01'}, 

[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'A', 'ipc_class': '62'}]
[{'ipc_section': 'B', 'ipc_class': '65'}, {'ipc_section': 'B', 'ipc_class': '25'}, {'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'G', 'ipc_class': '06'}, {'ipc_section': 'H', 'ipc_class': '04'}]
[{'ipc_section': 'G', 'ipc_class': '06'}]
[{'ipc_section': 'C', 'ipc_class': '12'}, {'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'F', 'ipc_class': '25'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'C', 'ipc_class': '08'}]
[{'ipc_section': 'G', 'ipc_class': '01'}]
[{'ipc_section': 'F', 'ipc_class': '04'}]
[{'ipc_section': 'F', 'ipc_class': '04'}]
[{'ipc_section': 'F', 'ipc_class': '04'

[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'B', 'ipc_class': '25'}]
[{'ipc_section': 'G', 'ipc_class': '01'}, {'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]
[{'ipc_section': 'B', 'ipc_class': '26'}]
[{'ipc_section': 'B', 'ipc_class': '26'}]
[{'ipc_section': 'E', 'ipc_class': '04'}]
[{'ipc_section': 'A', 'ipc_class': '21'}]
[{'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'A', 'ipc_class': '47'}]
[{'ipc_section': 'A', 'ipc_class': '47'}, {'ipc_section': 'A', 'ipc_class': '61'}, {'ipc_section': 'B', 'ipc_class': '65'}]
[{'ipc_section': 'H', 'ipc_class': '2'}, {'ipc_section': 'H', 'ipc_class': '3'}]
[{'ipc_section': 'A', 'ipc_class': '61'}]


In [47]:
df

,patent_number,patent_date,patent_processing_time,ipc_section,ipc_class,assignee_id,IPCs,wipos,marked
0,5387603,1995-02-07,433,NaN,NaN,5330279e-a21e-438c-80d1-7e6ae793c092,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
1,5771295,1998-06-23,552,NaN,NaN,2986e46e-7937-477f-9bc5-1f5eec05183f,"[{'ipc_section': 'H', 'ipc_class': '04'}]",[{'wipo_field_id': '2'}],1
2,5585587,1996-12-17,832,NaN,NaN,f7152515-d420-42d9-a7b6-3a2796bc5d10,"[{'ipc_section': 'G', 'ipc_class': '10'}]","[{'wipo_field_id': '34'}, {'wipo_field_id': '2...",1
3,5321490,1994-06-14,568,NaN,NaN,ec72750e-f789-4749-91f7-6292f7c6fda7,"[{'ipc_section': 'G', 'ipc_class': '01'}]","[{'wipo_field_id': '10'}, {'wipo_field_id': '1...",1
4,5750497,1998-05-12,1161,NaN,NaN,9a3afa40-66e5-4921-b831-64287f2052e0,"[{'ipc_section': 'C', 'ipc_class': '07'}, {'ip...","[{'wipo_field_id': '16'}, {'wipo_field_id': '1...",1
...,...,...,...,...,...,...,...,...,...
23905,10299691,2019-05-28,225,NaN,NaN,1b2d67d2-f1f9-4c29-8cd1-a5e75ba27be7,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...",[{'wipo_field_id': '13'}],1
23906,10292890,2019-05-21,966,NaN,NaN,f0997ba8-1ec2-4c0c-aaea-d80066a49317,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1
23907,10299968,2019-05-28,343,NaN,NaN,e5fc32c3-a913-455a-94ce-9932d30eefff,"[{'ipc_section': 'A', 'ipc_class': '61'}, {'ip...","[{'wipo_field_id': '13'}, {'wipo_field_id': '6'}]",1
23908,10149616,2018-12-11,2133,NaN,NaN,4eb39673-7f59-4c45-bf6c-2275dccf9048,"[{'ipc_section': 'A', 'ipc_class': '61'}]",[{'wipo_field_id': '13'}],1


In [25]:
k=0
while len(df) < 50000:
    token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
    token2 = '}&f=["patent_number","patent_title","patent_date","patent_kind","patent_type","assignee_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + str(L1[k:k+700]) + token2
    r = requests.get(url)
    html= r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["patents"]
    df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
    k+=700
    print(k)

700
1400
2100
2800
3500
4200
4900
5600
6300
7000
7700
8400
9100
9800
10500
11200
11900
12600
13300
14000
14700
15400
16100
16800
17500
18200
18900
19600
20300
21000
21700
22400
23100
23800
24500
25200
25900
26600


In [26]:
df

,patent_number,patent_title,patent_date,patent_kind,patent_type,assignees,marked
0,5387603,"1,5,7-trisubstituted indoline compounds and sa...",1995-02-07,A,utility,[{'assignee_id': '5330279e-a21e-438c-80d1-7e6a...,1.0
1,5771295,5-2-5 matrix system,1998-06-23,A,utility,[{'assignee_id': '2986e46e-7937-477f-9bc5-1f5e...,1.0
2,5585587,Acoustic image localization apparatus for dist...,1996-12-17,A,utility,[{'assignee_id': 'f7152515-d420-42d9-a7b6-3a27...,1.0
3,5321490,Active near-field object sensor and method emp...,1994-06-14,A,utility,[{'assignee_id': 'ec72750e-f789-4749-91f7-6292...,1.0
4,5750497,Acylated insulin,1998-05-12,A,utility,[{'assignee_id': '9a3afa40-66e5-4921-b831-6428...,1.0
...,...,...,...,...,...,...,...
694,5215272,Winding device having a tilting table and method,1993-06-01,A,utility,"[{'assignee_id': None, 'assignee_key_id': None}]",NaN
695,4034472,Wire termination apparatus,1977-07-12,A,utility,[{'assignee_id': '62769650-fc22-40db-a248-fb03...,NaN
696,10063472,Wireless communication terminal,2018-08-28,B2,utility,[{'assignee_id': '5718551a-fe57-489d-a94a-9286...,NaN
697,9933072,Working cylinder,2018-04-03,B2,utility,[{'assignee_id': 'fdb5f4af-1c7e-49cb-9716-cec9...,NaN


In [29]:
df = df.reset_index(drop=True)

In [30]:
df['marked'] = df['marked'].fillna(0)

In [36]:
max(df[df.marked==1].patent_date)

'2020-06-09'

In [32]:
df

,patent_number,patent_title,patent_date,patent_kind,marked,assignee_id
0,5387603,"1,5,7-trisubstituted indoline compounds and sa...",1995-02-07,A,1.0,5330279e-a21e-438c-80d1-7e6ae793c092
1,5771295,5-2-5 matrix system,1998-06-23,A,1.0,2986e46e-7937-477f-9bc5-1f5eec05183f
2,5585587,Acoustic image localization apparatus for dist...,1996-12-17,A,1.0,f7152515-d420-42d9-a7b6-3a2796bc5d10
3,5321490,Active near-field object sensor and method emp...,1994-06-14,A,1.0,ec72750e-f789-4749-91f7-6292f7c6fda7
4,5750497,Acylated insulin,1998-05-12,A,1.0,9a3afa40-66e5-4921-b831-64287f2052e0
...,...,...,...,...,...,...
50364,5215272,Winding device having a tilting table and method,1993-06-01,A,0.0,None
50365,4034472,Wire termination apparatus,1977-07-12,A,0.0,62769650-fc22-40db-a248-fb03540f65b3
50366,10063472,Wireless communication terminal,2018-08-28,B2,0.0,5718551a-fe57-489d-a94a-9286974683c0
50367,9933072,Working cylinder,2018-04-03,B2,0.0,fdb5f4af-1c7e-49cb-9716-cec98702c0c6


# Assignees df

In [75]:
df_assignees=pd.DataFrame(columns=["assignee_id", "assignee_organization","assignee_type", "assignee_lastknown_country", "assignee_total_num_patents", "assignee_total_num_inventors"])

In [76]:
#L_tmp = list(np.unique(df.assignee_id.dropna().values))
L2 = []
k=0
while k < len(assignees)-100:
    L2.append(json.dumps(assignees[k:k+100]))
    k+=100
L2.append(json.dumps(assignees[k:]))

In [77]:
for i in range(len(L2)):    
    token1 ='https://api.patentsview.org/assignees/query?q={"assignee_id":'
    token2 = '}&f=["assignee_id","assignee_organization","assignee_type","assignee_lastknown_country", "assignee_total_num_patents", "assignee_total_num_inventors"]&s=[{"assignee_organization":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + L2[i] + token2
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["assignees"]
    df_assignees = pd.concat([df_assignees,pd.read_json(json.dumps(Dict),orient='records')])
    print(i*100)
df_assignees=df_assignees.reset_index(drop=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [80]:
np.argmax(df_assignees.assignee_total_num_patents.values)

153

In [ ]:
df.to_csv('./info_patents.csv')
df_assignees.to_csv('./info_companies.csv')

In [82]:
df_assignees.iloc[153]

assignee_id                            ee0168d9-bf74-4513-bfbf-e73c933f90e7
assignee_organization           International Business Machines Corporation
assignee_type                                                             2
assignee_lastknown_country                                               US
assignee_total_num_patents                                           142160
assignee_total_num_inventors                                          63301
Name: 153, dtype: object